In [1]:
cd ..

C:\Users\agus\Dropbox\Doctorado\NeuroData


In [2]:
%matplotlib qt5
from copy import deepcopy
from importlib import reload
import time
import PyLeech.AbfExtension as abfe
import numpy as np
import matplotlib.pylab as plt
import PyLeech.sorting_with_python as swp
import PyLeech.spsortUtils
from sklearn.cluster import KMeans
import multiprocessing
import PyLeech.constants as constants
import PyLeech.filterUtils
import PyLeech.BurstUtils as BurstUtils

nan = constants.nan
opp0 = constants.opp0





np.set_printoptions(precision=3)
plt.ion()

filename = "RegistrosDP_PP/2018_11_06_0004.abf"
block = abfe.ExtendedAxonRawIo("RegistrosDP_PP/2018_11_06_0004.abf")
ch_info = block.ch_info

[print(ch, item) for ch, item in ch_info.items()]

n1 = np.array(block.getSignal_single_segment('IN5'))
n2 = np.array(block.getSignal_single_segment('IN6'))
fs = block.get_signal_sampling_rate()
data_len = int(len(n1)/2)
train_time = abfe.generateTimeVector(data_len, fs)


del block

# plt.figure()
# PyLeech.filterUtils.plotSpectrums(n1, n2, sampling_rate=fs, nperseg=10000)

n1_filt = PyLeech.filterUtils.runButterFilter(n1, 3000, sampling_rate=fs)
n2_filt = PyLeech.filterUtils.runButterFilter(n2, 3000, sampling_rate=fs)

n1_filt = PyLeech.filterUtils.runButterFilter(n1_filt, 20, sampling_rate=fs, butt_order=4, btype='high')
n2_filt = PyLeech.filterUtils.runButterFilter(n2_filt, 20, sampling_rate=fs, butt_order=4, btype='high')

# PyLeech.filterUtils.plotSpectrums(n2, n2_filt, sampling_rate=fs, nperseg=10000)

train_data = np.array(
    [n1_filt[:data_len],
     n2_filt[:data_len],
    ]
)
del n1, n2, n1_filt, n2_filt
train_time


sorter = PyLeech.spsortUtils.SpSorter(filename, train_data, train_time, fs)
del train_data, train_time

sorter.normTraces()
sorter.smoothAndFindPeaks(vect_len=7, threshold=6, min_dist=100)
sorter.makeEvents(149, 150)
sorter.makeNoiseEvents(size=4000)
sorter.getGoodEvents(threshold=25)

sorter.getPcaBase(plot_vectors=False)

sorter.getVectorWeights(40)
start = time.time()
sorter.KMeansClusterEvents(25, use_pca=True, dim_size=25, save=False)
end = time.time()
print(end-start)

Vm1 {'id': 0, 'sampling_rate': 20000.0, 'dtype': 'int16', 'units': 'mV', 'gain': 0.03051757880712104, 'offset': 0.0, 'group_id': 0, 'index': 0}
Im1 {'id': 1, 'sampling_rate': 20000.0, 'dtype': 'int16', 'units': 'nA', 'gain': 0.03051757880712104, 'offset': 0.0, 'group_id': 0, 'index': 1}
IN5 {'id': 5, 'sampling_rate': 20000.0, 'dtype': 'int16', 'units': 'V', 'gain': 0.00030517578125, 'offset': 0.0, 'group_id': 0, 'index': 2}
IN6 {'id': 6, 'sampling_rate': 20000.0, 'dtype': 'int16', 'units': 'V', 'gain': 0.00030517578125, 'offset': 0.0, 'group_id': 0, 'index': 3}


c:\users\agus\appdata\local\programs\python\python36\lib\site-packages\scipy\signal\_arraytools.py:45: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  b = a[a_slice]


Found 2625 out of 2733 events
0 -10753.29779456732
1 -3032.450422127018
2 -1886.9518229971036
3 -1377.5223869266483
4 -1138.5447022265598
5 -991.7210816771149
6 -867.9427395985549
7 -767.5874380236783
8 -676.5925632570797
9 -589.6949648441987
10 -505.66796508309017
11 -432.40803071106893
12 -362.31636694343615
13 -300.84603587409583
14 -246.31569425430644
15 -193.92376595190035
16 -142.03307642947402
17 -93.82495759414269
18 -51.53653847282294
19 -14.60642033523618
20 21.769258908214397
21 55.45830324599228
22 86.61416088205806
23 115.00383345700902
24 141.40471034536677
25 164.9370536566239
26 186.7647912885368
27 207.2216083446383
28 226.6453240700448
29 244.91976849148705
30 261.83029855235145
31 277.82422950820364
32 292.19449069458824
33 305.5811400933417
34 317.852095363236
35 329.1968597495161
36 339.9723742971346
37 349.7683935411096
38 359.16518686416384
39 367.62131631786906
Running
41.792710304260254


In [3]:
from sklearn import mixture

In [4]:
start = time.time()
evt_proy = np.dot(sorter.evts[sorter.good_evts], sorter.unitary[:,0:25])
gmm = mixture.GaussianMixture(n_components=20, covariance_type='full').fit(evt_proy)
end = time.time()
print('elapsed time: %i' % int(end - start))

elapsed time: 0


In [6]:
sorter.viewPcaClusters(go_pandas=True, clust_dim=10)

In [8]:
start = time.time()
sorter.KMeansClusterEvents(15, use_pca=True, dim_size=25, save=False)
end = time.time()
print('KMeans took %i' % int(end-start))

      

orig_clusters = deepcopy(sorter.clusters)
sorter.plotClusters()
      
      
start = time.time()
evt_proy = np.dot(sorter.evts[sorter.good_evts], sorter.unitary[:,0:25])
gmm = mixture.GaussianMixture(n_components=15, covariance_type='full', n_init=300, max_iter=200).fit(evt_proy)


sorter.clusters = gmm.predict(evt_proy)
end = time.time()
print('GMM took %i' % int(end - start))
sorter.plotClusters()

Running
KMeans took 22


c:\users\agus\appdata\local\programs\python\python36\lib\site-packages\matplotlib\pyplot.py:513: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


GMM took 281


c:\users\agus\appdata\local\programs\python\python36\lib\site-packages\matplotlib\pyplot.py:513: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
